# Things we won't do

* Load a structure with missing atoms
* Check for clashes or structural errors
* Support automatic filling in missing atoms/loops 

# User contracts

* Input data should have all the hydrogens. Complete molecules.
* Molecules can only become invalid through inplace modifications. Methods would not return invalid molecules.

# Reading from file

Reading biopolymer molecules from files, either in `sdf` or in `pdb` formats.

* from `sdf`: No residue information.
* from `pdb`: No detailed chemical information. Bond order, formal charges, stereochemistry, etc.

**Multiple molecules in single file**, so far. Molecule expected to be complete, including hydrogens.

* Load from SDF, perceive residues
* Load from PDB, fill in details according to known residues?

In [ ]:
protein = Molecule.from_file('some_mol.sdf')
protein.perceive_residues(pattern=['SMARTS_PATTERN', 'SMARTS_PATTERN2'])

dna_helix = Molecule.from_file('some_dna.sdf')
dna_helix.perceive_residues(nucleotides=True)
# Do we want to differentiate deoxy with ribonucleotides?

protein = Molecule.from_pdb_file('some_mol.pdb')

print(protein)
> Molecule with 5000 atoms

protein = Molecule.from_pdb_file('some_mol_missing_hs.pdb')
> IncompletePDBError: The Open Force Field Toolkit requires all 
> protons to be assigned before reading from PDB format. 
> (possibly more info here)

# Perceiving residues - Should be automatic for PDB
protein.perceive_residues()


Or we could have a special atom-typed object

In [ ]:
# Or a special atom-typed molecule
protein = PDBMolecule.from_pdb_file('some_mol.pdb')
print(protein)
> <PDBMolecule ...>

Or we could have a special method for atom-typed molecules/files

In [ ]:
# Or special method for PDB (atom-typed)
protein = Molecule.from_pdbfile('some_mol.pdb')
print(protein)
> <Molecule with X atoms>

# Reading from sequence

Reading biopolymer molecules from monomer/aa sequence string.

> - What about terminal caps? 

**Not much of a use for this**

    How to come up with conformers? Protonation? 
    How to match conformers to sequence?

In [ ]:
# from 3-letter code
protein = Molecule.from_sequence("AlaValGly", code='3-letter')

# from 1-letter code (default?)
protein = Molecule.from_sequence("AVQ")

print(protein)
> Molecule with 5000 atoms

# from 1-letter code (default?)
protein = Molecule.from_sequence("AVQZ")
> UnrecognizedAAError: Unrecognized amino acid in position 3.
# Analogous for 3-letter code. Could simply a ValueError do it?

# Histidines Protonation states

If we know the bonds orders we should be able to know the protonation state at a given pH and vice versa.

# Flexible bookkeeping and grouping

Some (many?) of these things can be accomplished using some selection algebra language, as many visualizers and packages do (e.g. `name CA and resn LYS` ). We could eventually have a `to_mdtraj` as a way to use its selection language capabilities.


## Atoms

In [ ]:
protein = Molecule.from_file('protein.sdf')
protein.atoms
> <generator object at ... >



## Residues

Residues in molecule should be iterable.

**NOTE:** Do we need a `Residue` type? Would the `Molecule` type be sufficient? 

In [ ]:
# From SDF
protein = Molecule.from_file('protein.sdf')
protein.atoms[10].metadata['residue_name']
> KeyError: Whats a residue?
protein.perceive_residues()
protein.atoms[10].metadata['residue_name']
> ALA
protein.atoms[10].metadata['residue_num']
> 2

# From PDB
protein_from_pdb = Molecule.from_pdb_file('protein.pdb')
protein_from_pdb.atoms[10].metadata['residue_name']
> ALA



* Support iteration over `Biopolymer.residues` when residues are separated by
    * resname/residue + resnum/resid

In [ ]:
protein.residues
> <generator object residues at 0x7f7268acecf0>

residues_list = []
for residue in protein.residues:
    residues_list.append(residue)

print(residues_list)
> [<Residue ABC, ID1>, ..., <Residue XYZ, IDN>]

In [ ]:
print(protein.hierarchy_schemes)
> [<HierarchyScheme with method_name='residues' uniqueness_criteria=['chain', 'residue_num', 'residue_name']>,
   <HierarchyScheme with method_name='chains' uniqueness_criteria=['chain']>]

protein.add_hierarchy_scheme(...)



We should expose only the setters defined by the hierarchy scheme

In [ ]:
# Using the hierarchy scheme above
protein.perceive_hierarchy(scheme=['residues'])
protein.residues[10]
> <HierarchyElement of type "residues" with id ('A','11','ALA') with residue_num 11, chain A, residue_name ALA>
protein.residues[('A','11','ALA')]
> <HierarchyElement of type "residues" with id ('A','11','ALA') with residue_num 11, chain A, residue_name ALA>



protein.residues[10].residue_name
> ALA
protein.residues[10].residue_num
> 2
protein.residues[10].chain
> 0
or 
> "A"
protein.residues[10].residue_name = "LYS"
protein.residues[10].residue_name
> LYS
protein.residues[10].residue_num = 5
protein.residues[10].residue_num
> 5


# Now Change the hierarchy scheme -- What does it do with the previous hierarchy metadata?
protein.perceive_hierarchy(scheme=['chains'])
[*protein.chains]
> SomeSortOfDictyList[<HierarchyElement of type "chains" with id "A" with chain A>]
protein.chains[0]
> <HierarchyElement of type "chains" with id "A" with chain A>
protein.chains['A']
> <HierarchyElement of type "chains" with id "A" with chain A>

protein.residues[10].chain = 'B'
protein.residues[10].chain
> "B"

# Option 1 -- Let's do this one!
protein.chains['B']
> KeyError
protein.perceive_hierarchy(scheme=['chains'])
protein.chains['B']
> <HierarchyElement of type "chains" with id "B" with chain B>

# Option 2 -- This is probably bad, because it means the hierarchies are silently being re-percieved in the
# background, which could screw with things that the user manually set
protein.chains['B']
> <HierarchyElement of type "chains" with id "B" with chain B>



protein.chains['A'].chain = 'B'
[*protein.chains]
> SomeSortOfDictyList[<HierarchyElement of type "chains" with id "B" with chain B>]



In [ ]:
protein.residues[10]
> <HierarchyElement of type "residues" with id ('A','11','ALA') with residue_num 11, chain A, residue_name ALA>

protein.residues[('A','11','ALA')]
> <HierarchyElement of type "residues" with id ('A','11','ALA') with residue_num 11, chain A, residue_name ALA>

protein.residues[10].chain
> A
protein.residues[10].chain='Z'
protein.residues[10].chain
> A
protein.residues[-1].chain
> Z

protein.residues[('A','11','ALA')]
> KeyError

protein.residues[10]
> <HierarchyElement of type "residues" with id ('A','12','HIS') with residue_num 12, chain A, residue_name HIS>

protein.residues[('Z','11','ALA')]
> <HierarchyElement of type "residues" with id ('Z','11','ALA') with residue_num 11, chain Z, residue_name ALA>
protein.residues[-1]
> <HierarchyElement of type "residues" with id ('Z','11','ALA') with residue_num 11, chain Z, residue_name ALA>


Drawbacks of the above approach
* It is dangerous to allow integer-based access to hierarchy-lists, since they can go out of date
    * Maybe only expose iterator?
    * Should we expose subscriptable access at all? (allow `protein.chains[('A',)]`?). Or only allow acces via things like `protein.select_residues`?
* It is dangerous to allow atom-level metadata assignment though HierarchyElements, since they can change the groupings
* 

Possible future ideas?
* Allow hierarchies within hierarchies? `protein.chains['A'].residues`?

### Nonstandard residue handling

* Detect unnatural AA and assign parameters, gracefully handle backbone interface w/ natural AAs
* Modified AA becomes a single residue
* Modified AA becomes several residues (maybe one for each monosaccharide)

In [ ]:
protein = Molecule.from_file('protein.sdf')
lipid = Molecule.from_file('lipid.sdf')

print(protein.atoms[10].metadata['resname'])
> KeyError: 

protein.perceive_residues(skip_hierarchy=True)
print(protein.atoms[10].metadata['resname'])
> ALA

protein.residues
> AttributeError

protein.perceive_hierarchy()  # Exposes iterables
protein.residues
> ALA ALA LYS GLU

protein_attachment_h = protein.mdtraj_sel('resname LYS and name HD2')[0]
lipid_attachment_h = lipid.mdtraj.sel('name H1')[0]
protein.delete_atom(protein_attachment_h)
lipid.delete_atom(lipid_attachment_h)
#add a new bond between the atoms that had their Hs removed
#protein.add_bond(protein_attachment_h-1, lipid_attachment_h-1) 

new_molecule = Molecule.merge_molecules(protein, lipid, protein_attachment_h, lipid_attachment_h, 
                                        bond_order=1, keep_metadata=False)

# Scheme is how to handle unknown regions of the molecule
# e.g. The modified region becomes a single unknown residue
new_molecule.perceive_hierarchy(scheme='default')


print([*new_molecule.residues])
> ALA, ALA, UNK, GLU

new_molecule.clear_hierarchy()
# Different scheme. The modified region tries to retain the original res label
new_molecule.perceive_hierarchy(scheme='separate_ptms')
print([*new_molecule.residues])
> ALA, ALA, LYS, LIP, GLU



# change atom properties/data
protein.atoms[10].metadata['resname'] = 'GLU'
protein.atoms[10].update_metadata(key='resname', value='GLU')

Merge_molecules should optionally return a mapping from the original atom inidces to the new ones

In [ ]:
new_molecule, mol1_to_new_mol_idxs, mol2_to_new_mol_idx2 = Molecule.merge_molecules(protein, 
                                                                                    lipid, 
                                                                                    protein_attachment_h, 
                                                                                    lipid_attachment_h, 
                                                                                    bond_order=1, 
                                                                                    keep_metadata=False, 
                                                                                    return_atom_maps=True)

Get residues given IDs

In [ ]:
protein.select_residues(resid=23)
> [<Residue GLY, 23>]

# Do we want to be able to get groups of residues? (what about getting atoms from these?)
protein.select_residues(resid=[23, 72, 12])
> [<Residue GLY, 23>, <Residue GLY, 23>, <Residue GLY, 23>]  # Maybe this should be a generator

Get the atoms from a given residue ID. Then the `Atom` must have some `resid` attribute?

In [ ]:
protein.select_residues(resid=23).atoms
> [<Atom name='' atomic number='6' resid='23'>, <Atom name='' atomic number='7' resid='23'>, 
   ..., <Atom name='' atomic number='1' resid='23'>]

# Do we want to be able to get groups of residues? (what about getting atoms from these?)
protein.select_residues(resid=[23, 72, 12]).atoms
> [
    [<Atom name='' atomic number='6' resid='23'>, ..., <Atom name='' atomic number='1' resid='23'>],
    [<Atom name='' atomic number='6' resid='72'>, ..., <Atom name='' atomic number='1' resid='72'>],
    [<Atom name='' atomic number='6' resid='12'>, ..., <Atom name='' atomic number='1' resid='12'>]
]

Get all residues matching a name

In [ ]:
protein.select_residues(name="LYS")
> [<Residue LYS, 3>, <Residue LYS, 21>, ..., <Residue LYS, 344>]

Renaming/transforming residues. 

> Should be just a metadata change

In [ ]:
protein.select_residues(resid=23)[0].name
> "GLY"
protein.select_residues(resid=23)[0].name = "LYS"
protein.select_residues(resid=23)[0].name
> "LYS"
protein.select_residues(resid=23)
> [<Residue LYS, 23>]
protein.select_residues(resid=23)[0].name = "WAT"
> 
protein.select_residues(resid=23)[0].name 
> "WAT"

# Do we want to be able to rename groups of residues?
# Then we need to match shapes/lengths -- is it getting out of control?
protein.select_residues(resid=[23, 72, 12]).name
> ["GLY", "ARG", "TYR"]
protein.select_residues(resid=[23, 72, 12]).name = ["LYS", "VAL", "CYS"]
protein.select_residues(resid=[23, 72, 12]).name
> ["LYS", "VAL", "CYS"]

We should be able to see numbers/ids and atoms

In [ ]:
protein.select_residues(resid=23)[0].

## Chains/Segments

We want to be able to group or cluster chains.

> Do we want a `Chain` class?

Assuming that you have multiple chains in your input files.

Chains should be iterable.

In [ ]:
protein = Molecule.from_file('protein.sdf')
protein.chains
> AttributeError: ...
protein.perceive_chains()
protein.chains
> ['A', 'B', ...,  'Z', '0', ..., '9']
protein.group_by(key='chains', values=['A', 'B'])
> AtomGroup

In [ ]:
protein = Molecule.from_file('protein.sdf')
protein.atoms[10].metadata['chain']
> 'B'

## Example. Protein - DNA merging

We expect coordinates of both molecules to be in the same frame of reference.

* Attach a cofactor like heme, which connects to 4 other residues

In [ ]:
# Load structures and perceive info
protein = Molecule.from_file('protein.sdf')
dna = Molecule.from_file('dna.sdf')
protein.perceive_residues()
protein.residues
> ALA ALA LYS ALA
dna.perceive_residues(nucleotides=True)
dna.residues
> A T C G
# Where they attach
protein_attachment = protein.mdtraj_sel('resname LYS and name HD2')[0]
dna_attachment = dna.chemical_environment_matches('c1ccn([H:1])c1')[0]
# Merge the two
new_molecule = Molecule.merge_molecules(protein, dna, protein_attachment, dna_attachment, 
                                        bond_order=1, keep_metadata=True)
                                        # What metadata to keep then?
new_molecule.residues
> ALA ALA LYS ALA A T C G
new_molecule.residue(5)  # Selects atoms in 6th residue in iter
> T

new_molecule.perceive_residues(amino_acids=True, 
                               nucleotides=True, 
                               clear_existing=True,
                               scheme='default')
new_molecule.residues
> ALA ALA UNK ALA T C G
or
> ALA ALA UNK T ALA C G

What if we want no residue information on DNA

In [ ]:
# Load structures and perceive info - No residues for DNA
protein = Molecule.from_file('protein.sdf')
dna = Molecule.from_file('dna.sdf')
protein.perceive_residues()
protein.residues
> ALA ALA LYS ALA
dna.residues
> AttributeError
# Where they attach
protein_attachment = protein.mdtraj_sel('resname LYS and name HD2')[0]
dna_attachment = dna.chemical_environment_matches('c1ccn([H:1])c1')[0]
# Merge the two
new_molecule = Molecule.merge_molecules(protein, dna, protein_attachment, dna_attachment, 
                                        bond_order=1, keep_metadata=True)
                                        # What metadata to keep then?
new_molecule.residues
> ALA ALA LYS ALA   # Note the difference here
new_molecule.residue(5)  # Selects atoms in 6th residue in iter
> IndexError: ...
new_molecule.residue(3)
> ALA

new_molecule.perceive_residues(amino_acids=True, 
                               nucleotides=True, 
                               clear_existing=True,
                               scheme='default')
new_molecule.residues
> ALA ALA UNK ALA T C G
or
> ALA ALA UNK T ALA C G


If we have multiple attachment sites do we just run `merge_molecules` multiple times?

# Modifications

## Adding/removing atoms

* Change protonation state of a residue

In [ ]:
mol = Molecule.from_file('protein.sdf')
mol.perceive_residues()
mol.residue(6)
> Residue ALA / AtomGroup with 15 atoms with index 90-105
new_idx = mol.add_atom(atomic_number=99, 
                       formal_charge=0, 
                       sterochemistry=None)
mol.add_bond(100, new_idx, bond_order=1, stereochemistry=None)
# EITHER
mol.remove_bond(100, 101)
mol.remove_atom(101)
# OR
mol.remove_atom(101)

mol.residue(6)
> Residue ALA / AtomGroup with 14 atoms

mol.perceive_residues()
mol.residue(6)
> Residue UNK / AtomGroup with 15 atoms



## Adding removing/residues

# Handling bonds

# Handling `pydantic` errors

In [5]:
from pydantic import BaseModel, ValidationError, validator


class UserModel(BaseModel):
    name: str
    username: str
    password1: str
    password2: str

    @validator('name')
    def name_must_contain_space(cls, v):
        if ' ' not in v:
            raise ValueError('must contain a space')
        return v.title()

    @validator('password2')
    def passwords_match(cls, v, values, **kwargs):
        if 'password1' in values and v != values['password1']:
            raise ValueError('passwords do not match')
        return v

    @validator('username')
    def username_alphanumeric(cls, v):
        assert v.isalnum(), 'must be alphanumeric'
        return v
    
    @classmethod
    def from_sequence(cls, seq):
        name = ' '.join(seq)
        if 'GLY' in seq:
            raise ValueError("GLY doesn't exist")
        return cls(name=name, username='blah', password1='foo', password2='bar')


user = UserModel(
    name='samuel colvin',
    username='scolvin',
    password1='zxcvbn',
    password2='zxcvbn',
)
print(user)
#> name='Samuel Colvin' username='scolvin' password1='zxcvbn' password2='zxcvbn'

try:
    protein = UserModel.from_sequence("AlaValGLY")

    #UserModel(
    #    name='samuel',
    #    username='scolvin',
    #    password1='zxcvbn',
    #    password2='zxcvbn2',
    #)
except ValidationError as e:
    print(e)
    """
    2 validation errors for UserModel
    name
      must contain a space (type=value_error)
    password2
      passwords do not match (type=value_error)
    """

name='Samuel Colvin' username='scolvin' password1='zxcvbn' password2='zxcvbn'


ValueError: GLY doesn't exist